In [9]:
import pandas as pd
from geopy.geocoders import GoogleV3
from tqdm import tqdm  # Import tqdm

def get_geocode(address):
    geolocator = GoogleV3(api_key='')
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('ADDRESS2.csv')

# Create empty lists to store latitude, longitude, and ZIP code
latitudes = []
longitudes = []
zip_codes = []

# Initialize the geolocator
geolocator = GoogleV3(api_key='')

# Use tqdm to add a progress bar
for _, row in tqdm(df.iterrows(), total=len(df)):  # tqdm(df.iterrows(), total=len(df)) replaces df.iterrows()
    address = f"{row['Address']}, Arlington, VA"  # Concatenate "Arlington, VA" to the address
    # Get latitude and longitude
    lat, lon = get_geocode(address)
    latitudes.append(lat)
    longitudes.append(lon)
    # Get ZIP code
    zip_code = None
    
    zip_codes.append(zip_code)

# Add latitude, longitude, and ZIP code columns to the DataFrame
df['Latitude'] = latitudes
df['Longitude'] = longitudes
df['ZIP_Code'] = zip_codes

# Save the DataFrame to a new CSV file
df.to_csv('Addresses2_with_Geocodes2.csv', index=False)


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:16<00:00,  1.55it/s]


In [8]:
import requests
from concurrent.futures import ThreadPoolExecutor

def get_zipcode(latitude_longitude):
    latitude, longitude = latitude_longitude
    api_key = ''  # Replace with your actual API key
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == 'OK':
        for result in data['results']:
            for component in result['address_components']:
                if 'postal_code' in component['types']:
                    return component['long_name']
    else:
        print('Error:', data['status'])
        return None

def add_zipcode(df):
    latitude_longitude_list = [(row['latitude'], row['longitude']) for index, row in df.iterrows()]
    with ThreadPoolExecutor() as executor:
        zipcodes = list(tqdm(executor.map(get_zipcode, latitude_longitude_list), total=len(latitude_longitude_list), desc="Processing"))
    df['zipcode'] = zipcodes
    return df

# Read CSV file containing latitude and longitude
input_csv_file = 'input.txt'
df = pd.read_csv(input_csv_file)

# Add zip codes to the dataframe
df_with_zip = add_zipcode(df)

# Export dataframe to CSV
output_csv_file = 'output.csv'
df_with_zip.to_csv(output_csv_file, index=False)

print(f'CSV with zip codes saved to {output_csv_file}')



Processing: 100%|████████████████████████████████████████████████████████████████████| 608/608 [00:42<00:00, 14.39it/s]

CSV with zip codes saved to output3.csv
